In [1]:
# Import Modules
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score



C:\Users\Joanna\anaconda3\envs\my_python\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# SQLAlchemy connectable
# DEFAULT engine = create_engine("postgresql://USERNAME:%s@HOST/mydatabase" % quote_plus("Password"))
engine = create_engine("postgresql://postgres:%s@localhost/Energy_Output_Expenses"% quote_plus("Password"))
engine.connect()


In [3]:
energy_consumption_df = pd.read_sql_table('energy_efficient',engine)
energy_consumption_df.head()

,doeid,dwcycle,tvtype1,typetherm,lgtinled,smartmeter,solar,elecveh,totalbtu,totaldol
0,100001,2,1,1,2,0,0,0,144647.71,2656.89
1,100002,2,1,1,2,0,-2,0,28034.61,975.00
2,100003,2,2,1,0,0,-2,0,30749.71,522.65
3,100004,1,2,1,4,1,0,0,86765.19,2061.77
4,100005,2,1,2,3,-4,-2,0,59126.93,1463.04


In [4]:
# Check the column names in your DataFrame
print(energy_consumption_df.columns)

# Drop any rows with missing values
energy_consumption_df.dropna(inplace=True)

# Drop the correct column 
column_to_drop = 'doeid'
if column_to_drop in energy_consumption_df.columns:
    energy_consumption_df.drop(column_to_drop, axis=1, inplace=True)
    print(f"{column_to_drop} column dropped successfully.")
else:
    print(f"Column {column_to_drop} not found in the DataFrame.")

# Print the updated DataFrame shape to confirm changes
print(energy_consumption_df.shape)


Index(['doeid', 'dwcycle', 'tvtype1', 'typetherm', 'lgtinled', 'smartmeter',
       'solar', 'elecveh', 'totalbtu', 'totaldol'],
      dtype='object')
doeid column dropped successfully.
(18496, 9)


In [5]:
energy_consumption_df.head()

,dwcycle,tvtype1,typetherm,lgtinled,smartmeter,solar,elecveh,totalbtu,totaldol
0,2,1,1,2,0,0,0,144647.71,2656.89
1,2,1,1,2,0,-2,0,28034.61,975.00
2,2,2,1,0,0,-2,0,30749.71,522.65
3,1,2,1,4,1,0,0,86765.19,2061.77
4,2,1,2,3,-4,-2,0,59126.93,1463.04


In [6]:
#Checking nulls
energy_consumption_df.isnull().sum()

dwcycle       0
tvtype1       0
typetherm     0
lgtinled      0
smartmeter    0
solar         0
elecveh       0
totalbtu      0
totaldol      0
dtype: int64

In [7]:
# Define a function to perform binning on TOTALBTU column
def bin_total_btu(total_btu):
    if total_btu < 55000:
        return 'Low'
    elif total_btu >= 55000 and total_btu < 95000:
        return 'Medium'
    else:
        return 'High'

# Apply binning function to create a new column 'BTU_Bin'
energy_consumption_df['BTU_Bin'] = energy_consumption_df['totalbtu'].apply(bin_total_btu)

# Perform one-hot encoding on the 'BTU_Bin' column
X = pd.get_dummies(energy_consumption_df.drop(['totalbtu'], axis=1), columns=['BTU_Bin'], drop_first=True)
y = energy_consumption_df['totalbtu']

In [8]:
# Check the balance of bins
bin_counts = energy_consumption_df['BTU_Bin'].value_counts()

# Display the counts of samples in each bin
print("Counts of samples in each bin:")
print(bin_counts)

Counts of samples in each bin:
BTU_Bin
High      6240
Low       6194
Medium    6062
Name: count, dtype: int64


In [9]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert feature names to strings
X_train.columns = X_train.columns.map(str)
X_test.columns = X_test.columns.map(str)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the Linear Regression model
lr_model = LinearRegression()

# Train the model
lr_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = lr_model.predict(X_test_scaled)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")


Mean Squared Error (MSE): 526618178.5795278
R-squared (R2): 0.7933460682175177
